# Advanced Usage: The `MorphingWorkflow` Classes

This notebook demonstrates the advanced, step-by-step approach to morphing EPW files using the specialized `MorphingWorkflow` classes (`MorphingWorkflowGlobal` and `MorphingWorkflowEurope`).

While the `morph_epw_*` functions are great for direct, one-shot tasks, the workflow classes are designed for complex projects where you need full control over **filename parsing, custom renaming, and process validation**. They enforce a safe, multi-step process that allows you to review and confirm each stage before executing the time-consuming morphing computation.

This notebook will focus on **`MorphingWorkflowGlobal`**.

## The Three-Step Workflow

The class is designed to be used as a state machine, guiding you through a logical sequence:

1.  **`map_categories()`**: Analyze the input filenames to extract meaningful data (like city, building type, etc.).
2.  **`configure_and_preview()`**: Define all execution parameters, validate them, and generate a "dry run" plan of how the final files will be named and organized.
3.  **`execute_morphing()`**: Run the final computation, confident that your plan and configuration are correct.

## General Setup

First, we'll import the necessary class and set up the paths for our files. 

**Important:** You must change the `jar_path` variable to the correct location of the `FutureWeatherGenerator_v3.0.1.jar` file on your system. You must also ensure that the EPW files exist at the specified paths.

In [1]:
import os
import pandas as pd
from pyfwg import MorphingWorkflowGlobal, DEFAULT_GLOBAL_GCMS

# --- Configuration ---
# !!! IMPORTANT: You MUST change this path to the correct location on your PC !!!
jar_path = r"D:\OneDrive - Universidad de Cádiz (uca.es)\Programas\FutureWeatherGenerator_v3.0.1.jar"

# --- Define file paths for the examples ---
pattern_epw_dir = 'epws/w_pattern'
keyword_epw_dir = 'epws/wo_pattern'

## Step 1: Map Categories from Filenames

This is the first and most critical step for organizing your workflow. The `map_categories` method analyzes your source filenames and extracts meaningful data that will be used later for renaming the output files.

In [2]:
# Instantiate a workflow object for this example
workflow = MorphingWorkflowGlobal()

Let's have a look at the filenames of the EPWs we are going to work with. First, the set that has a pattern we can define with regex:

In [3]:
# Define the list of files for this specific case
epw_files_with_pattern = [os.path.join(pattern_epw_dir, f) for f in os.listdir(pattern_epw_dir) if f.endswith('.epw')]
print(epw_files_with_pattern)

['epws/w_pattern\\london_uhi-type-2.epw', 'epws/w_pattern\\sevilla_uhi-type-1.epw']


And second, the set that does not have a pattern:

In [4]:
# Define the list of files for this specific case
epw_files_without_pattern = [os.path.join(keyword_epw_dir, f) for f in os.listdir(keyword_epw_dir) if f.endswith('.epw')]
print(epw_files_without_pattern)

['epws/wo_pattern\\GBR_London.Gatwick.037760_IWEC_uhi_type-2.epw', 'epws/wo_pattern\\sevilla_in_this_one_the_uhi_is_type-1.epw']


We can also provide a `keyword_mapping` dictionary to translate the extracted raw values (like `sevilla`) into a clean, final format (like `Seville`).

In [5]:
# Define the mapping rules to categorize filenames based on their content.
# This dictionary provides the logic for both keyword searching and normalization.
mapping_rules = {
    # The top-level keys ('city', 'uhi') define the final category names
    # that will be available as placeholders (e.g., {city}) in the output filename.
    'city': {
        # The second-level keys ('Seville', 'London') are the final, clean values
        # that will be assigned to the 'city' category.
        'Seville': ['sevilla', 'SVQ'],  # The keywords to search for (case-insensitive).
        'London': ['london', 'gatwick'] # A list is used for multiple possible keywords.
    },
    'uhi': {
        # For convenience, if there is only one keyword to search for,
        # you can provide it as a single string instead of a list with one item.
        'type-1': 'type-1',
        'type-2': 'type-2'
    }
}

# For example, if a filename contains 'SVQ', pyfwg will assign the value 'Seville'
# to the 'city' category for that file.

### Example 1.1: Using a Regex Pattern with Normalization

If the filenames follow a consistent pattern, you can use the `input_filename_pattern` argument.

In [6]:
workflow.map_categories(
    epw_files=epw_files_with_pattern,
    # This pattern extracts raw values like 'sevilla' and 'type-1'
    input_filename_pattern=r'(?P<city>.*?)_(?P<uhi>.*)',
    # This dictionary then normalizes them to 'Seville' and 'type-1'. In case of type-1, we want to keep it as it is, so we use the same value.
    keyword_mapping=mapping_rules
)

2025-09-23 10:41:34 - INFO - --- Step 1: Mapping categories from filenames ---
2025-09-23 10:41:34 - INFO - Mapped 'epws/w_pattern\london_uhi-type-2.epw': {'city': 'London', 'uhi': 'uhi-type-2'}
2025-09-23 10:41:34 - INFO - Mapped 'epws/w_pattern\sevilla_uhi-type-1.epw': {'city': 'Seville', 'uhi': 'uhi-type-1'}
2025-09-23 10:41:34 - INFO - Category mapping complete.


The mapped categories are saved to the attribute `.epw_categories` as a dictionary. Let's view it as a DataFrame for better visualization:

In [7]:
pd.DataFrame(workflow.epw_categories)

,epws/w_pattern\london_uhi-type-2.epw,epws/w_pattern\sevilla_uhi-type-1.epw
city,London,Seville
uhi,uhi-type-2,uhi-type-1


### Example 1.2: Using Keyword-Only Search

If your files are irregularly named, set `input_filename_pattern` to `None`. `pyfwg` will then search for the keywords from your mapping dictionary anywhere in the filename.

In [8]:
workflow.map_categories(
    epw_files=epw_files_without_pattern,
    input_filename_pattern=None,
    keyword_mapping=mapping_rules
)

2025-09-23 10:41:43 - INFO - --- Step 1: Mapping categories from filenames ---
2025-09-23 10:41:43 - INFO - Mapped 'epws/wo_pattern\GBR_London.Gatwick.037760_IWEC_uhi_type-2.epw': {'city': 'London', 'uhi': 'type-2'}
2025-09-23 10:41:43 - INFO - Mapped 'epws/wo_pattern\sevilla_in_this_one_the_uhi_is_type-1.epw': {'city': 'Seville', 'uhi': 'type-1'}
2025-09-23 10:41:43 - INFO - Category mapping complete.


Let's view the newly mapped categories:

In [9]:
pd.DataFrame(workflow.epw_categories)

,epws/wo_pattern\GBR_London.Gatwick.037760_IWEC_uhi_type-2.epw,epws/wo_pattern\sevilla_in_this_one_the_uhi_is_type-1.epw
city,London,Seville
uhi,type-2,type-1


## Step 2: Configure and Preview the Plan

This is the combined configuration and preview step. Here, you define all the parameters for the FutureWeatherGenerator tool and the output filenames. The method validates everything and then shows you a "dry run" plan of the final results.

The `output_filename_pattern` is very powerful. It can use placeholders from your mapped categories (like `{city}`) and also placeholders for any of the `fwg_` parameters (like `{fwg_interpolation_method_id}`).

In [10]:
# We will continue with the 'workflow' object, which now contains the keyword-mapped files.
workflow.configure_and_preview(
    final_output_dir='./final_results_workflow',
    # The placeholders {city} and {uhi} match the mapping rule keys
    output_filename_pattern='{city}_{uhi}_{ssp}_{year}_interp-{fwg_interpolation_method_id}',
    # The {ssp} placeholder will be populated from this mapping
    # For instance, the SSP scenario will be shown as 'SSP2-4.5' in the output filename instead of 'ssp245'.
    scenario_mapping={'ssp245': 'SSP2-4.5', 'ssp585': 'SSP5-8.5'},
    
    # --- FWG Configuration ---
    fwg_jar_path=jar_path,
    fwg_gcms=['BCC_CSM2_MR'], # Use just one GCM for a quick test
    fwg_interpolation_method_id=2, # This value will appear in the filename
    fwg_show_tool_output=True,
    delete_temp_files=False # Set to False for debugging
)

2025-09-23 10:43:31 - INFO - --- Step 2: Configuring and Previewing Morphing Plan ---



          MORPHING CONFIGURATION & PREVIEW
  - FWG JAR Path: D:\OneDrive - Universidad de Cádiz (uca.es)\Programas\FutureWeatherGenerator_v3.0.1.jar
  - Final Output Directory: D:\Python\pyfwg\pyfwg\tutorials\final_results_workflow
  - EPWs to be Morphed (2 files):
    - GBR_London.Gatwick.037760_IWEC_uhi_type-2.epw
    - sevilla_in_this_one_the_uhi_is_type-1.epw

  For input file: GBR_London.Gatwick.037760_IWEC_uhi_type-2.epw
    -> Generated 'ssp126_2050.epw' will be moved to: D:\Python\pyfwg\pyfwg\tutorials\final_results_workflow\London_type-2_ssp126_2050_interp-2.epw
    -> Generated 'ssp245_2050.epw' will be moved to: D:\Python\pyfwg\pyfwg\tutorials\final_results_workflow\London_type-2_SSP2-4.5_2050_interp-2.epw
    -> Generated 'ssp370_2050.epw' will be moved to: D:\Python\pyfwg\pyfwg\tutorials\final_results_workflow\London_type-2_ssp370_2050_interp-2.epw
    -> Generated 'ssp585_2050.epw' will be moved to: D:\Python\pyfwg\pyfwg\tutorials\final_results_workflow\London_type-2_SSP

You can also inspect the rename plan just shown above in the attribute `.rename_plan`

In [13]:
workflow.rename_plan

{'epws/wo_pattern\\GBR_London.Gatwick.037760_IWEC_uhi_type-2.epw': {'ssp126_2050.epw': './final_results_workflow\\London_type-2_ssp126_2050_interp-2.epw',
  'ssp245_2050.epw': './final_results_workflow\\London_type-2_SSP2-4.5_2050_interp-2.epw',
  'ssp370_2050.epw': './final_results_workflow\\London_type-2_ssp370_2050_interp-2.epw',
  'ssp585_2050.epw': './final_results_workflow\\London_type-2_SSP5-8.5_2050_interp-2.epw',
  'ssp126_2080.epw': './final_results_workflow\\London_type-2_ssp126_2080_interp-2.epw',
  'ssp245_2080.epw': './final_results_workflow\\London_type-2_SSP2-4.5_2080_interp-2.epw',
  'ssp370_2080.epw': './final_results_workflow\\London_type-2_ssp370_2080_interp-2.epw',
  'ssp585_2080.epw': './final_results_workflow\\London_type-2_SSP5-8.5_2080_interp-2.epw'},
 'epws/wo_pattern\\sevilla_in_this_one_the_uhi_is_type-1.epw': {'ssp126_2050.epw': './final_results_workflow\\Seville_type-1_ssp126_2050_interp-2.epw',
  'ssp245_2050.epw': './final_results_workflow\\Seville_type-

In [11]:
workflow.inputs

{'epw_files': ['epws/wo_pattern\\GBR_London.Gatwick.037760_IWEC_uhi_type-2.epw',
  'epws/wo_pattern\\sevilla_in_this_one_the_uhi_is_type-1.epw'],
 'final_output_dir': './final_results_workflow',
 'output_filename_pattern': '{city}_{uhi}_{ssp}_{year}_interp-{fwg_interpolation_method_id}',
 'scenario_mapping': {'ssp245': 'SSP2-4.5', 'ssp585': 'SSP5-8.5'},
 'fwg_jar_path': 'D:\\OneDrive - Universidad de Cádiz (uca.es)\\Programas\\FutureWeatherGenerator_v3.0.1.jar',
 'run_incomplete_files': False,
 'delete_temp_files': False,
 'temp_base_dir': './morphing_temp_results',
 'show_tool_output': True,
 'fwg_params': {'gcms': ['BCC_CSM2_MR'],
  'create_ensemble': True,
  'winter_sd_shift': 0.0,
  'summer_sd_shift': 0.0,
  'month_transition_hours': 72,
  'use_multithreading': True,
  'interpolation_method_id': 2,
  'limit_variables': True,
  'solar_hour_adjustment': 1,
  'diffuse_irradiation_model': 1,
  'add_uhi': True,
  'epw_original_lcz': 14,
  'target_uhi_lcz': 1},
 'fwg_params_formatted': {

In [12]:
workflow.inputs['fwg_params']

{'gcms': ['BCC_CSM2_MR'],
 'create_ensemble': True,
 'winter_sd_shift': 0.0,
 'summer_sd_shift': 0.0,
 'month_transition_hours': 72,
 'use_multithreading': True,
 'interpolation_method_id': 2,
 'limit_variables': True,
 'solar_hour_adjustment': 1,
 'diffuse_irradiation_model': 1,
 'add_uhi': True,
 'epw_original_lcz': 14,
 'target_uhi_lcz': 1}

## Step 3: Execute the Morphing Process

This is the final step. The `execute_morphing` method takes **no arguments**. It acts as the "Go" button, running the entire process based on the configuration from the previous step.

As a safeguard, it will raise an error if the configuration is invalid, so it's good practice to check the `is_config_valid` flag first.

In [ ]:
workflow.execute_morphing()

In [ ]:
print("Script finished. Uncomment the final lines to execute the morphing.")